# Similarity Search

## Level 2

We benchmark the algorithms based on the time it takes to index images and locate the most similar image based on its features using the Caltech-101 dataset. We also experiment with t-SNE and PCA.

### Understanding the time it takes to index images and locate the most similar image based on its features

For these experiments we will use the features of the Caltech101 dataset that we read above.

First, let's choose a random image to experiment with. We will be using the same image for all the following experiments.

In [ ]:
import numpy as np
import pickle
from tqdm import tqdm, tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors

import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [ ]:
filenames = pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('data/features-caltech101-resnet.pickle',
                                'rb'))
class_ids = pickle.load(open('data/class_ids-caltech101.pickle', 'rb'))

In [ ]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

In [ ]:
random_image_index = random.randint(0, num_images)

### Standard features

The following experiments are based on the ResNet-50 features derived from the images of the Caltech101 dataset. 

### Standard features + Brute Force Algorithm on one image

We will be timing the indexing for various Nearest Neighbors algorithms, so let's start with timing the indexing for the Brute force algorithm. While running terminal commands in iPython like the `timeit` command, the variables are not stored in memory, so we need to rerun the same command to compute and store the results in the variable. 

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)

Now, let's look at the time it takes to search for the nearest neighbors for the selected random image using the trained model with the Brute force algorithm.

In [ ]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

###  Standard features + k-d Tree Algorithm  on one image

Now let's turn our attention to the next nearest neighbors algorithm, the k-d tree. Let's time the indexing for the k-d tree algorithm.

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)

Now, time the search for the same random image using the k-d tree trained model.

In [ ]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

###  Standard features + Ball Tree Algorithm  on one image

Finally, its time for our last nearest neighbors algorithm - the Ball Tree algorithm. As before, let's calculate the time it takes to train the model.

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)

As before, let's time the search for the Ball Tree model.

In [ ]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

We will increase the number of our test images so that we can experiment with how the scalability of different nearest neighbors algorithms change. Let's choose a random set of 100 or 1000 images to experiment. 


Generate a list of images to do the next set of experiments on.

In [ ]:
random_image_indices = random.sample(range(0, num_images), 1000)
random_feature_list = [
    feature_list[each_index] for each_index in random_image_indices
]

### Standard features + Brute Force Algorithm on a set of images

Time the search for the Brute force algorithm.

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

### Standard features +  k-d Tree Algorithm on a set of images

Time the search for the k-d tree algorithm.

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

### Standard features +  Ball Tree Algorithm on a set of images

Time the search for the Ball Tree algorithm.

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

### PCA

Now we have seen the time it takes to index and search using nearest neighbor algorithms on the full feature length. We can use PCA to compress the features and reduce the time. As before we set the number of features intended.

In [ ]:
num_feature_dimensions = 100
num_feature_dimensions = min(num_images, num_feature_dimensions,
                             len(feature_list[0]))

Train the PCA model with the number of desired feature dimensions.

In [ ]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()

Let's try to understand the importance of each of the resultant features. The numbers displayed below show the relative importance of the first 20 features.

In [ ]:
print(pca.explained_variance_ratio_[0:20])

Repeat the timing experiments. We use the same random image to experiment.

### PCA + Brute Force Algorithm on one image

Let's time the indexing for the brute force algorithm.

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)

We will now time the search for the brute force algorithm.

In [ ]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

###  PCA + k-d Tree Algorithm  on one image

Time the indexing for the k-d tree algorithm.

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)

Time the search for the k-d tree algorithm.

In [ ]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

###  PCA + Ball Tree Algorithm  on one image

Time the indexing for the ball tree algorithm.

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)

Time the search for the ball tree algorithm.

In [ ]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

We use the same random indices to experiment.

Generate a list of images to do the next set of experiments on.

In [ ]:
random_feature_list_compressed = [
    feature_list_compressed[each_index] for each_index in random_image_indices
]

### PCA + Brute Force Algorithm on a set of images

Time the search for the brute force algorithm.

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

### PCA + k-d Tree Algorithm on a set of images

Time the search for the k-d tree algorithm.

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

### PCA + Ball Tree Algorithm on a set of images

Time the search for the Ball Tree algorithm.

In [ ]:
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

### Annoy 

We must have `annoy` installed. We can install it using pip, `pip3 install annoy`.

In [ ]:
from annoy import AnnoyIndex

In [ ]:
# Time the indexing for Annoy
t = AnnoyIndex(2048)  # Length of item vector that will be indexed
starttime = time.time()
for i in range(num_images):
    feature = feature_list[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

### Annoy on one image 

Time the search for one image for Annoy.

In [ ]:
u = AnnoyIndex(2048)
%timeit u.get_nns_by_vector(feature_list[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list[random_image_index],
                              5,
                              include_distances=True)

Helper function to time the search for multiple images for Annoy. Perform the search for the same image multiple times to get an average value.


In [ ]:
def calculate_annoy_time():
    for i in range(0, 100):
        indexes = u.get_nns_by_vector(feature_list[random_image_index],
                                      5,
                                      include_distances=True)

### Annoy on a set of images

Time the search for multiple images for Annoy.

In [ ]:
%time calculate_annoy_time()

### PCA + Annoy

Now, let's time the indexing for Annoy for the PCA generated features.

In [ ]:
starttime = time.time()
# Length of item vector that will be indexed
t = AnnoyIndex(num_feature_dimensions)

for i in range(num_images):
    feature = feature_list_compressed[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

### PCA + Annoy for one image

Time the search for one image for Annoy.

In [ ]:
u = AnnoyIndex(num_feature_dimensions)
%timeit u.get_nns_by_vector(feature_list_compressed[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                              5,
                              include_distances=True)

Helper function to time the search for multiple images for Annoy. Perform the search for the same image multiple times to get an average value.


In [ ]:
def calculate_annoy_time():
    for i in range(0, 100):
        indexes = u.get_nns_by_vector(
            feature_list_compressed[random_image_index],
            5,
            include_distances=True)

### PCA + Annoy on a set of images

Time the search for multiple images for Annoy.

In [ ]:
%time calculate_annoy_time()

### NMS Lib

In [ ]:
import nmslib

In [ ]:
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(feature_list_compressed)
index.createIndex({'post': 2}, print_progress=True)

### NMS Lib on one image

In [ ]:
# Query for the nearest neighbors of the first datapoint
%timeit index.knnQuery(feature_list_compressed[random_image_index], k=5)
ids, distances = index.knnQuery(feature_list_compressed[random_image_index],
                                k=5)

### NMS Lib on a set of images

In [ ]:
# Get all nearest neighbors for all the datapoint
# using a pool of 4 threads to compute
%timeit index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)
neighbors = index.knnQueryBatch(feature_list_compressed, k=5, num_threads=16)

### Falconn


In [ ]:
import falconn

In [ ]:
# Setup different parameters for Falonn
parameters = falconn.LSHConstructionParameters()
num_tables = 1
parameters.l = num_tables
parameters.dimension = num_feature_dimensions
parameters.distance_function = falconn.DistanceFunction.EuclideanSquared
parameters.lsh_family = falconn.LSHFamily.CrossPolytope
parameters.num_rotations = 1
parameters.num_setup_threads = 1
parameters.storage_hash_table = falconn.StorageHashTable.BitPackedFlatHashTable

# Train the Falconn model
falconn.compute_number_of_hash_functions(16, parameters)

### Falconn on a set of images

In [ ]:
dataset = np.array(feature_list_compressed)
a = np.random.randn(8677, 100)
a /= np.linalg.norm(a, axis=1).reshape(-1, 1)
dataset = a

index = falconn.LSHIndex(parameters)
%time index.setup(dataset)

query_object = index.construct_query_object()
num_probes = 1
query_object.set_num_probes(num_probes)

searchQuery = np.array(feature_list_compressed[random_image_index])
searchQuery = a[0]
%timeit query_object.find_k_nearest_neighbors(searchQuery, 5)

### PCA + Annoy

In [ ]:
# Time the indexing for Annoy for the PCA generated features
starttime = time.time()
# Length of item vector that will be indexed
t = AnnoyIndex(num_feature_dimensions)

for i in range(num_images):
    feature = dataset[i]
    t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(40)  # 50 trees
t.save('data/caltech101index.ann')

In [ ]:
u = AnnoyIndex(num_feature_dimensions)
# Time the search for one image for Annoy
%timeit u.get_nns_by_vector(dataset[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(dataset[random_image_index],
                              5,
                              include_distances=True)